# Maximizing Induced Cardinality Under a Determinantal Point Process (Google)



더 다양한 아이템 들을 가진 집합, 유사 아이템이 더 적게 유사한 집합에 더 높은 확률을 부여한다.    
추천 퀄리티가 더 높은 아이템을 더 선호하면서도 동시에 다양한 아이템을 추천하여 사용자 경험을 풍부하게 만들어 주는 추천 시스템

## 학습 데이터 
전체 아이템 집합 [n]에서 k개 아이템을 추천 하려는 시나리오를 가정한다 
- [S_1, S_2, ..., S_r] : 이전에 추천된 집합
- [E_1, E_2, ..., E_r] : 추천된 집합 S에서 사용자가 실제 클릭하거나 시청 혹은 청취한 아이템 집합

목표 : 사용자의 과거 상호작용을 바탕으로 가장 적합한 k개 아이템을 추천 해야 한다

In [2]:
import numpy as np

n = 5 # total items
d = 3 # item feature vector dims

# each item's feature vector 
B = np.random.rand(n, d) # shape : (nxd)
B 

array([[0.15668538, 0.93828152, 0.81556315],
       [0.22422434, 0.98614922, 0.77689908],
       [0.60243815, 0.38002305, 0.21655129],
       [0.9999247 , 0.26342654, 0.68503416],
       [0.72259087, 0.51220051, 0.42268556]])

In [3]:
# training params
theta = np.random.rand(d) # shape : (d,)
theta_diag = np.diag(theta) # shape : (d x d)
theta_diag

array([[0.7305593 , 0.        , 0.        ],
       [0.        , 0.01170129, 0.        ],
       [0.        , 0.        , 0.95230851]])

In [4]:
def compute_Lij(i, j):
    """커널 매트릭스 L의 Lij를 계산하는 함수

    Args:
        i (_type_): i번째 아이템 인덱스
        j (_type_): j번째 아이템 인덱스
    """

    bi = B[i]
    bj = B[j]

    return np.dot(np.dot(bi, theta_diag), bj)

compute_Lij(0, 1)

0.6398859847030931

In [5]:
L = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        L[i,j] = compute_Lij(i, j)

print("Kernel Matrix L :\n", L)

Kernel Matrix L :
 [[0.66165852 0.63988598 0.24132058 0.64939546 0.41662324]
 [0.63988598 0.62289633 0.26328502 0.67365747 0.43700031]
 [0.24132058 0.26328502 0.31149101 0.58252528 0.40746977]
 [0.64939546 0.67365747 0.58252528 1.17815281 0.80517934]
 [0.41662324 0.43700031 0.40746977 0.80517934 0.55466466]]


In [7]:
"""
    f(S) = Σ_E⊆S ( |E| * det(LE) / det(LS + I) ) = f(S) = Tr(I - (LS + I)^(-1))
    의 Greedy Optimization

    1. 초기 집합 `S`를 빈 집합으로 설정합니다.
    2. 가능한 모든 아이템 i에 대해, `f(S ∪ {i}) - f(S)`를 계산하여 가장 큰 변화를 주는 아이템을 찾습니다.
    3. 이 아이템을 `S`에 추가합니다.
    4. 모든 아이템이 선택되거나 더 이상 향상시킬 수 없을 때까지 2번과 3번을 반복합니다.
"""



'f(S) = Σ_E⊆S ( |E| * det(LE) / det(LS + I) ) = f(S) = Tr(I - (LS + I)^(-1)) \n'

In [14]:

def f_hat(S, B, m):
    return len(S) - (len(S)/m) - ((np.trace(B[S]))/m**2) - ((np.trace(np.dot(B[S], B[S].T)))/m**3)

def greedy_optimization(items, B, m):
    S = []

    S.append(0)
    items.remove(0)
    while items:
        # Find item that maximizes the increase in f_hat(S)
        i_max = max(items, key=lambda i: f_hat(S+[i], B, m) - f_hat(S, B, m))

        # Get maximum possitble increase
        max_increase = f_hat(S + [i_max], B, m) - f_hat(S, B, m)

        # If maximum possible increase is non-positive, stop
        if max_increase <= 0:
            break

        # Otherwise, add item to S and remove from items
        S.append(i_max)
        items.remove(i_max)

    return S

items = list(range(10)) # recommendable items

B = np.random.rand(10, 10) # item's feature vector
m = 5 # ?


S_optimal = greedy_optimization(items, B, m)
S_optimal



[0, 5, 4, 9, 3, 6, 7, 8, 1, 2]

PSD를 set of M-matrices로 projecting 하는 법

In [4]:
import numpy as np

def create_matrix_L(n):
    """In real-world, this matrix must be contructed using user-item interactions

    Args:
        n (_type_): _description_
    """

    L = np.random.rand(n, n)
    L = (L + L.T) / 2 # ensure PSD(positive semi-definite)

    return L

n = 10
L = create_matrix_L(n)
L

array([[0.52413366, 0.46066763, 0.93011811, 0.61540564, 0.32046804,
        0.45242995, 0.68450215, 0.64236375, 0.70601802, 0.2663018 ],
       [0.46066763, 0.31963438, 0.37765636, 0.49963099, 0.53973885,
        0.14620751, 0.34568554, 0.13100718, 0.65685419, 0.47794374],
       [0.93011811, 0.37765636, 0.50606527, 0.20157441, 0.50601024,
        0.25127244, 0.44161949, 0.54161512, 0.4852199 , 0.53876692],
       [0.61540564, 0.49963099, 0.20157441, 0.79677119, 0.53803812,
        0.79455912, 0.76739929, 0.78284015, 0.70347282, 0.4058268 ],
       [0.32046804, 0.53973885, 0.50601024, 0.53803812, 0.71598164,
        0.84010157, 0.39311391, 0.5226099 , 0.70573156, 0.43553763],
       [0.45242995, 0.14620751, 0.25127244, 0.79455912, 0.84010157,
        0.03613128, 0.32823945, 0.63692758, 0.54894676, 0.23353589],
       [0.68450215, 0.34568554, 0.44161949, 0.76739929, 0.39311391,
        0.32823945, 0.16593451, 0.32240764, 0.39055368, 0.44060899],
       [0.64236375, 0.13100718, 0.5416151

In [6]:
4.1 Projecting to the set of M-matrices
We tried several methods for projecting to the set of (real, symmetric) PSD M-matrices for the PIC
method. We found that flipping the signs of any positive off-diagonal elements, then projecting to
the PSD cone by truncating negative eigenvalues at zero worked best. If the PSD projection resulted
in any positive off-diagonal elements, we simply iterated the process of flipping their signs and
projecting to the PSD cone until the resulting matrix satisfied all requirements.
Note that the sign-flipping step computes a projection onto the set of Z-matrices (under the Frobenius
norm). Since the set of Z-matrices is closed and convex, as is the set of PSD matrices, this means
that the iterative process described above is guaranteed to converge. (Though it will not necessarily
converge to the projection onto the intersection of the two convex sets.)

여기서 만드는 M-matrices를 만드는 파이썬 코드를 작성해줘

SyntaxError: invalid syntax (2642163214.py, line 1)